# Get quicker (first) plot 

by using [PackageCompiler.jl](https://github.com/JuliaLang/PackageCompiler.jl)

This file contains instructions for how to create a Julia sysimage file (to speed up the plots, CSV and DataFrames) that can be used for either working in the REPL or in IJulia.

## Step 1: Create a File with some (slow) Commands

Put the contents of the next cell in a file `precompile_PS.jl`. It's just some common command for plotting and DataFrames. 

You need the file `Data/Options_prices_US_Canada.csv` for this to run. It's available in the repository.

In [ ]:
using Plots

p = plot(rand(2,2))
scatter!(rand(3),rand(3))
display(p)

using Dates, CSV, DataFrames
DataFile = "Data/Options_prices_US_Canada.csv"
df1 = CSV.read(DataFile,DataFrame,normalizenames=true,dateformat="mm/dd/yy")

df1.date       .+= Dates.Year(2000)     #03/30/17 to 03/30/2017
df1.expiration .+= Dates.Year(2000)

select!(df1,Not([:exchange,:option_symbol,:style,:unadjusted]))  #deleting some columns
rename!(df1,:adjusted_close => :close)          #renaming a column
show(df1)

## Step 2: Create a Sysimage

Run the following (in the REPL) and wait a few minutes:
```
using PackageCompiler
create_sysimage(["Plots","CSV","DataFrames"], sysimage_path="sys_PS.so", precompile_execution_file="precompile_PS.jl")
```

## Step 3: Enjoy the Speed Up

Restart the Julia REPL as

```
julia --sysimage D:/Julia/sys_PS.so
```
from the command line (change the folder as needed). Perhaps you want to create a desktop link for this.

Now, run some Julia code (with plots and CSV/DataFrames) and notice that it is fast. For instance, rerun the Julia code in `precompile_PS.jl`.

## IJulia and VS code

To run IJulia, first install the kernel (change the folder as needed) by running the following in the REPL
```
using IJulia
sysimage = "D:/Julia/sys_PS.so"
installkernel("Julia_sys_PS","--sysimage=$(sysimage)")
```

Now you can start jupyter as usual and select the kernel and perhaps run my Plots tutorial. You may have to change the kernel once you have opened an existing notebook.

For VS code, see https://www.julia-vscode.org/docs/dev/userguide/compilesysimage/

## A Final Remark

Notice that if you want to take advantage of updated packages, then you need to recreate the sysimage.